# Fe K $\alpha$ line from SN1006

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m_p = 938.272e6 #eV
m_e = 0.511e6  #eV
c = 3e10 #cm/s
hPlanck = 4.135e-15 #eV s

In [3]:
def velocity(E_kin, rest_mass):
    E_kin = E_kin
    rest_mass = rest_mass
    gamma = 1+(E_kin/rest_mass)
    beta = np.sqrt(1-(1/(gamma**2)))
    v = c*beta
    return v, beta, gamma

## CR spectrum

$J_{CR}(T_i) = A_i(T_i^2 + 2T_im_ic^2)^{-\frac{\delta}{2}}exp\left(-\frac{T_i}{T_i^c}\right)$
where $A_i$ is the normalisation factor ($\mathrm{eV^{-1}cm^{-2}s^{-1}sr^{-1}}$), $T_i$ is the kinetic energy of the CR particle of species $i$, $T_i^c$ is the cut-off energy, $m_i$ is the mass of the CR particle $i$, all of these quantities in $\mathrm{GeV}$ and $\delta$ is the power-law index.

In [4]:
def J_CRp(T_p, del_p, Ap, Tp_c):
    # Gives spectrum in eV-1 cm-2 s-1 sr-1
    return Ap * np.power((T_p ** 2) + (2 * T_p * m_p), -del_p / 2) * np.exp(- T_p / Tp_c)

def J_CRe(T_e, del_e, Ae, Te_c):
    # Gives spectrum in eV-1 cm-2 s-1 sr-1
    return Ae * np.power((T_e ** 2) + (2 * T_e * m_e), -del_e / 2) * np.exp(- T_e / Te_c)

## Multi-wavelength fit

In order to find the normalisation and cut-off values for the CR particle spectra, we fit multiwavelength data (radio, gamma) from the SN1006 region.

Radio: synchrotron emissions from electrons

$\Phi_{syn} = \frac{\sqrt{3}}{2\pi}\frac{e^3B_0}{m_ec^2}\frac{1}{\hbar{E}}\int_0^{\infty}N_{CRe}(T_e)R\left(\frac{T_e}{E_c(T_e)}\right)\,dT_e$

Gamma: 
pion decay from protons

$\Phi_{\gamma, p}(E_\gamma) = 4\pi{n_H}\int\frac{d\sigma}{dE_\gamma}(T_p, E_\gamma)N_{CRp}(T_p)\,dT_p$ 

inverse Compton and relativistic Bremsstrahlung from electrons 

$\Phi_{\gamma, e}(E_\gamma) = cn_H\int\sigma_{brem}(E_\gamma, T_e)N_{CRe}(T_e)\,dT_e + \int\frac{dN_{iso}}{d{\omega}dt}(T_e, E_\gamma)N_{CRe}(T_e)\,dT_e$


In [5]:
from CRp_gamma import Phi_pp_gamma 
from CRe_gamma_radio import Phi_syn, Phi_e_rel_brem, Phi_e_IC

ImportError: cannot import name 'Phi_pp_gamma' from 'CRp_gamma' (/Users/ravikularaman/VScode/SN1006/sn-1006/CRp_gamma.py)

In [6]:
def N_CRp(T_p, del_p, Ap, Tp_c):
    # Gives particle density in eV-1 cm-3
    v_p, _, _ = velocity(T_p, m_p)
    return (4 * np.pi / v_p) * J_CRp(T_p, del_p, Ap, Tp_c)

def N_CRe(T_e, del_e, Ae, Te_c):
    # Gives particle density in eV-1 cm-3
    v_e, _, _ = velocity(T_e, m_e)
    return (4 * np.pi / v_e) * J_CRp(T_e, del_e, Ae, Te_c)

Loading data

In [7]:
radio_data = np.genfromtxt(open("Data txt\SN1006_data_radio.txt", "r"), delimiter=',')
xray_data = np.genfromtxt(open("Data txt\SN1006_data_suzaku.txt", "r"), delimiter=',')
fermi_data = np.genfromtxt(open("Data txt\SN1006_data_SW_FERMI.txt", "r"), delimiter=',')
fermi_upper_data = np.genfromtxt(open("Data txt\SN1006_data_SW_upper_FERMI_Acero2015.txt", "r"), delimiter=',')
hess_data = np.genfromtxt(open("Data txt\SN1006_data_SW_HESS.txt", "r"), delimiter=',')

FileNotFoundError: [Errno 2] No such file or directory: 'Data txt\\SN1006_data_radio.txt'

Converting all to units $\mathrm{eV}$ and $\mathrm{erg.cm^{-2}.s^{-1}}$

In [ ]:
Er, Phi_radio = radio_data[:, 0] * hPlanck, 1e-23 * radio_data[:, 0] * radio_data[:, 1]
Ex, Phi_xray = xray_data[:, 0], xray_data[:, 1] * 1.602e-12
Ef, Efm, Efp, Phi_fermi, Phi_fp, Phi_fm = fermi_data[:, 0] * 1e9, fermi_data[:, 1] * 1e9, fermi_data[:, 2] * 1e9, fermi_data[:, 3], fermi_data[:, 4], fermi_data[:, 5]
Efu, Phi_fermi_up = fermi_upper_data[:, 0], fermi_upper_data[:, 1]
Eh, Phi_hp, Phi_hm = hess_data[:, 0], hess_data[:, 1], hess_data[:, 2] 

Plot data

In [ ]:
plt.plot(Er, Phi_radio, 'y^')
plt.plot(Ex, Phi_xray, 'bs')
plt.errorbar(Ef, Phi_fermi, yerr=[Phi_fermi-Phi_fm,Phi_fp-Phi_fermi], xerr=[Ef-Efm,Efp-Ef], fmt='ko')
plt.errorbar(Efu, Phi_fermi_up, yerr=0.5 * Phi_fermi_up, linestyle='none', marker='_', color="green", uplims=True)
plt.fill_between(Eh, Phi_hm, Phi_hp, color=[(236.0/255.0,92.0/255.0,92.0/255.0)])
plt.set_xscale('log')
plt.set_yscale('log')
plt.legend()
plt.set_xlabel(r'$E_{\gamma} \textrm{ (eV)}$',fontsize=fs)
plt.set_ylabel(r'$E^2_\gamma\phi(E_\gamma) \textrm{ (erg cm$^{-2}$ s$^{-1}$)}$',fontsize=fs)
plt.set_xlim(1.0e-7,1.0e14)
plt.set_ylim(1.0e-15,1.0e-10)
plt.show()